# NLP for stock market prediction

In [1]:
import time
start = time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack, csr_matrix
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
import eli5


from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from sklearn import preprocessing


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import gc

/opt/conda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not b

ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/stocknews/RedditNews.csv
/kaggle/input/stocknews/Combined_News_DJIA.csv
/kaggle/input/stocknews/DJIA_table.csv


## 1. Load the data

In [3]:
df = pd.read_csv("../input/stocknews/Combined_News_DJIA.csv",low_memory=False,
                    parse_dates=["Date"])

full_stock = pd.read_csv("../input/stocknews/DJIA_table.csv",low_memory=False,
                    parse_dates=["Date"])

#add the closing stock value to the df - this will be the y variable
df["Close"]=full_stock.Close

#save the index
all_index = df.index
#y1 = train["Label"].copy()
#y2 = train["Diff"].copy()

#show how the dataset looks like
df.head(5)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",17949.369141
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",17929.990234
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",17694.679688
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,17409.720703
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,"b""Georgia confict could set back Russia's 

## 2. Data cleaning

### NA treatment
We'll simply fill the NAs in the numerical features (Date, Close)

In [4]:
#check for NAN
df.isnull().sum()

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
Close    0
dtype: int64

There are a few headlines missing. Let's fill them in with a whitespace.

In [5]:
df = df.replace(np.nan, ' ', regex=True)

#sanity check
df.isnull().sum().sum()

0

### Remove the HTML tags
There are several non-word tags in the headlines that would just bias the sentiment analysis so we need to remove them and replace with ''. This can be done with regex.

In [6]:
test = df.copy()
test = test.replace('b\"|b\'|\\\\|\\\"', '', regex=True)
test.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Close
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...,BREAKING: Musharraf to be impeached.',Russia Today: Columns of troops roll into Sout...,Russian tanks are moving towards the capital o...,"Afghan children raped with 'impunity,' U.N. of...",150 Russian tanks have entered South Ossetia w...,"Breaking: Georgia invades South Ossetia, Russi...",The 'enemy combatent' trials are nothing but a...,...,Al-Qaeda Faces Islamist Backlash',Condoleezza Rice: The US would not act to prev...,This is a busy day: The European Union has ap...,"Georgia will withdraw 1,000 soldiers from Iraq...",Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis: Georgia invades South Osse...,Indian shoe manufactory - And again in a seri...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico's Kidnapping Surge,17949.369141
1,2008-08-11,1,Why wont America and Nato help us? If they won...,Bush puts foot down on Georgian conflict',Jewish Georgian minister: Thanks to Israeli tr...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks 'faked',What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S.Ossetia blames...,...,"Do not believe TV, neither Russian nor Georgia...",Riots are still going on in Montreal (Canada) ...,China to overtake US as largest manufacturer',War in South Ossetia [PICS]',Israeli Physicians Group Condemns State Torture',Russia has just beaten the United States over...,Perhaps *the* question about the Georgia - Rus...,Russia is so much better at war',So this is what it's come to: trading sex for ...,17929.990234


It looks like it's working so now we'll apply the same regex to the dataset.

In [7]:
df = df.replace('b\"|b\'|\\\\|\\\"', '', regex=True)
df.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Close
0,2008-08-08,0,Georgia 'downs two Russian warplanes' as count...,BREAKING: Musharraf to be impeached.',Russia Today: Columns of troops roll into Sout...,Russian tanks are moving towards the capital o...,"Afghan children raped with 'impunity,' U.N. of...",150 Russian tanks have entered South Ossetia w...,"Breaking: Georgia invades South Ossetia, Russi...",The 'enemy combatent' trials are nothing but a...,...,Al-Qaeda Faces Islamist Backlash',Condoleezza Rice: The US would not act to prev...,This is a busy day: The European Union has ap...,"Georgia will withdraw 1,000 soldiers from Iraq...",Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis: Georgia invades South Osse...,Indian shoe manufactory - And again in a seri...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico's Kidnapping Surge,17949.369141
1,2008-08-11,1,Why wont America and Nato help us? If they won...,Bush puts foot down on Georgian conflict',Jewish Georgian minister: Thanks to Israeli tr...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks 'faked',What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S.Ossetia blames...,...,"Do not believe TV, neither Russian nor Georgia...",Riots are still going on in Montreal (Canada) ...,China to overtake US as largest manufacturer',War in South Ossetia [PICS]',Israeli Physicians Group Condemns State Torture',Russia has just beaten the United States over...,Perhaps *the* question about the Georgia - Rus...,Russia is so much better at war',So this is what it's come to: trading sex for ...,17929.990234


## 3. Sentiment analysis
Now I run the sentiment analysis extracting the compound score that goes from -0.5 (most negative) to 0.5 (most positive). I'm going to use the "dirty" texts in this part because VADER can utilize the information such as ALL CAPS, punctuation, etc. I'll also calculate the subjectivity of each headline using the TextBlob package.

Initialise the VADER analyzer.

In [8]:
Anakin = SentimentIntensityAnalyzer()

Anakin.polarity_scores(" ")

{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

Write a function to save the subjectivity score directly from TextBlob function's output. Subjectivity score might detect direct quotes in the headlines and positive stuff is rarely quoted in the headline.

In [9]:
def detect_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

detect_subjectivity(" ") #should return 0

0.0

In [10]:
cols = []
for i in range(1,26):
    col = ("Top{}".format(i))
    cols.append(col)

In [11]:
start_vect=time.time()
print("ANAKIN: 'Intializing the process..'")

#get the name of the headline columns
cols = []
for i in range(1,26):
    col = ("Top{}".format(i))
    cols.append(col)


for col in cols:
    df[col] = df[col].astype(str) # Make sure data is treated as a string
    df[col+'_comp']= df[col].apply(lambda x:Anakin.polarity_scores(x)['compound'])
    df[col+'_sub'] = df[col].apply(detect_subjectivity)
    print("{} Done".format(col))
    
print("VADER: Vaderization completed after %0.2f Minutes"%((time.time() - start_vect)/60))

ANAKIN: 'Intializing the process..'
Top1 Done
Top2 Done
Top3 Done
Top4 Done
Top5 Done
Top6 Done
Top7 Done
Top8 Done
Top9 Done
Top10 Done
Top11 Done
Top12 Done
Top13 Done
Top14 Done
Top15 Done
Top16 Done
Top17 Done
Top18 Done
Top19 Done
Top20 Done
Top21 Done
Top22 Done
Top23 Done
Top24 Done
Top25 Done
VADER: Vaderization completed after 0.75 Minutes


In [12]:
#the text isn't required anymore
df = df.drop(cols,axis=1)
df.head(5)

,Date,Label,Close,Top1_comp,Top1_sub,Top2_comp,Top2_sub,Top3_comp,Top3_sub,Top4_comp,...,Top21_comp,Top21_sub,Top22_comp,Top22_sub,Top23_comp,Top23_sub,Top24_comp,Top24_sub,Top25_comp,Top25_sub
0,2008-08-08,0,17949.369141,-0.5994,0.0,0.0000,0.000000,-0.3612,0.000000,-0.7089,...,-0.7579,0.666667,-0.6249,0.0,-0.2755,0.00,-0.8519,0.200000,0.1280,0.0
1,2008-08-11,1,17929.990234,0.8156,0.0,-0.3182,0.288889,0.4404,0.100000,-0.1965,...,-0.8020,0.000000,0.0000,0.0,-0.3182,0.00,-0.1832,0.500000,0.0000,0.0
2,2008-08-12,0,17694.679688,0.0258,1.0,0.0000,0.000000,-0.7845,0.833333,-0.6124,...,-0.5994,0.000000,0.5267,0.0,0.3818,0.35,0.0000,0.454545,0.0000,0.0
3,2008-08-13,0,17409.720703,-0.7184,0.0,-0.8074,0.000000,-0.6369,0.000000,-0.1280,...,-0.2960,0.000000,0.4939,0.0,-0.5719,0.00,-0.4215,0.100000,-0.3400,0.0
4,2008-08-14,1,17140.240234,0.2023,0.0,-0.5994,0.000000,0.6808,0.400000,-0.8689,...,-0.4404,0.000000,-0.5994,0.0,0.1779,0.00,-0.6908,0.500000,0.7096,0.0


Summarise the compound and subjectivity scores weighted by rating of the headline (top1 has the most weight)

In [13]:
comp_cols = []
for col in cols:
    comp_col = col + "_comp"
    comp_cols.append(comp_col)

w = np.arange(1,26,1).tolist()
w.reverse()

weighted_comp = []
max_comp = []
min_comp = []
for i in range(0,len(df)):
    a = df.loc[i,comp_cols].tolist()
    weighted_comp.append(np.average(a, weights=w))
    max_comp.append(max(a))
    min_comp.append(min(a))

df['compound_mean'] = weighted_comp
df['compound_max'] = max_comp
df['compound_min'] = min_comp


sub_cols = []
for col in cols:
    sub_col = col + "_sub"
    sub_cols.append(sub_col)


weighted_sub = []
max_sub = []
min_sub = []
for i in range(0,len(df)):
    a = df.loc[i,sub_cols].tolist()
    weighted_sub.append(np.average(a, weights=w))
    max_sub.append(max(a))
    min_sub.append(min(a))

df['subjectivity_mean'] = weighted_sub
df['subjectivity_max'] = max_sub
df['subjectivity_min'] = min_sub

to_drop = comp_cols+sub_cols
df = df.drop(to_drop, axis=1)

In [14]:
df.head(5)

,Date,Label,Close,compound_mean,compound_max,compound_min,subjectivity_mean,subjectivity_max,subjectivity_min
0,2008-08-08,0,17949.369141,-0.350337,0.2144,-0.9260,0.163685,0.666667,0.0
1,2008-08-11,1,17929.990234,-0.085277,0.8156,-0.8271,0.202921,0.720000,0.0
2,2008-08-12,0,17694.679688,-0.318394,0.5423,-0.8591,0.374076,1.000000,0.0
3,2008-08-13,0,17409.720703,-0.162032,0.5106,-0.8074,0.176371,0.900000,0.0
4,2008-08-14,1,17140.240234,-0.194879,0.7177,-0.8689,0.319615,1.000000,0.0


## 4. Lag the timeseries features
To allow the models to look into the past, we'll add features which are essentially just copies of rows from n-steps back. In order to not create too many new features we'll add only features from 1 week prior to the current datapoint.


We'll first try this out on a copy of the dataset.

In [15]:
to_lag = ["compound_mean", "compound_max", "compound_min", "subjectivity_mean",  "subjectivity_max", "subjectivity_min"]
#adding lag columns from one day up to a week back
for col in to_lag:
    for i in range(1, 8):
        new_name = col + ("_lag_{}".format(i))
        df[new_name] = df[col].shift(i)
        
df.tail(3)

,Date,Label,Close,compound_mean,compound_max,compound_min,subjectivity_mean,subjectivity_max,subjectivity_min,compound_mean_lag_1,...,subjectivity_max_lag_5,subjectivity_max_lag_6,subjectivity_max_lag_7,subjectivity_min_lag_1,subjectivity_min_lag_2,subjectivity_min_lag_3,subjectivity_min_lag_4,subjectivity_min_lag_5,subjectivity_min_lag_6,subjectivity_min_lag_7
1986,2016-06-29,1,11642.469727,-0.170199,0.7783,-0.9211,0.179425,0.90625,0.0,-0.027175,...,0.95,0.70,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1987,2016-06-30,1,11782.349609,-0.235306,0.7351,-0.9578,0.186443,0.70000,0.0,-0.170199,...,1.00,0.95,0.70,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1988,2016-07-01,1,11734.320312,-0.291085,0.5719,-0.9618,0.220872,0.90000,0.0,-0.235306,...,0.75,1.00,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In this process, rows with NAs were created. Unfortunately these rows will have to be removed since we simply don't have the data from the future.

In [16]:
#Show many rows need to be removed
df.head(10) 

,Date,Label,Close,compound_mean,compound_max,compound_min,subjectivity_mean,subjectivity_max,subjectivity_min,compound_mean_lag_1,...,subjectivity_max_lag_5,subjectivity_max_lag_6,subjectivity_max_lag_7,subjectivity_min_lag_1,subjectivity_min_lag_2,subjectivity_min_lag_3,subjectivity_min_lag_4,subjectivity_min_lag_5,subjectivity_min_lag_6,subjectivity_min_lag_7
0,2008-08-08,0,17949.369141,-0.350337,0.2144,-0.9260,0.163685,0.666667,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-08-11,1,17929.990234,-0.085277,0.8156,-0.8271,0.202921,0.720000,0.0,-0.350337,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-08-12,0,17694.679688,-0.318394,0.5423,-0.8591,0.374076,1.000000,0.0,-0.085277,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,2008-08-13,0,17409.720703,-0.162032,0.5106,-0.8074,0.176371,0.900000,0.0,-0.318394,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2008-08-14,1,17140.240234,-0.194879,0.7177,-0.8689,0.319615,1.000000,0.0,-0.162032,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
5,2008-08-15,1,17400.750000,-0.143104,0.4404,-0.7481,0.227282,0.900000,0.0,-0.194879,...,0.666667,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
6,2008-08-18,0,18011.070312,-0.263546,0.5106,-0.9260,0.216935,1.000000,0.0,-0.143104,...,0.720000,0.666667,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
7,2008-08-19,0,17780.830078,-0.360490,0.5719,-0.8720,0.256786,1.000000,0.0,-0.263546,...,1.000000,0.720000,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2008-08-20,1,17829.730469,-0.231969,0.4588,-0.8807,0.095403,1.000000,0.0,-0.360490,...,0.900000,1.000000,0.720000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2008-08-21,1,17804.869141,-0.268522,0.5719,-0.9022,0.107994,1.000000,0.0,-0.231969,...,1.000000,0.900000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Above we can see that the first 7 rows now have missing values. Let's delete them and reset index.

In [17]:
df = df.drop(df.index[[range(0,7)]])
df = df.reset_index(drop=True)

#sanity check for NaNs
df.isnull().sum().sum()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/base.py:4291: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



0

In [19]:
df.head(5)

,Date,Label,Close,compound_mean,compound_max,compound_min,subjectivity_mean,subjectivity_max,subjectivity_min,compound_mean_lag_1,...,subjectivity_max_lag_5,subjectivity_max_lag_6,subjectivity_max_lag_7,subjectivity_min_lag_1,subjectivity_min_lag_2,subjectivity_min_lag_3,subjectivity_min_lag_4,subjectivity_min_lag_5,subjectivity_min_lag_6,subjectivity_min_lag_7
0,2008-08-19,0,17780.830078,-0.360490,0.5719,-0.8720,0.256786,1.000000,0.0,-0.263546,...,1.0,0.72,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2008-08-20,1,17829.730469,-0.231969,0.4588,-0.8807,0.095403,1.000000,0.0,-0.360490,...,0.9,1.00,0.720000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2008-08-21,1,17804.869141,-0.268522,0.5719,-0.9022,0.107994,1.000000,0.0,-0.231969,...,1.0,0.90,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2008-08-22,1,17675.160156,-0.072582,0.6369,-0.8979,0.165417,0.666667,0.0,-0.268522,...,0.9,1.00,0.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2008-08-25,0,17733.099609,-0.433257,0.0000,-0.8769,0.347709,0.850000,0.0,-0.072582,...,1.0,0.90,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_param = {
    "sublinear_tf":True,
    "strip_accents":'unicode',
    "stop_words":"english",
    "analyzer":'word',
    "token_pattern":r'\w{1,}',
    #"ngram_range":(1,1),
    "dtype":np.float32,
    "norm":'l2',
    "min_df":5,
    "max_df":.9,
    "smooth_idf":True
}

lemm = WordNetLemmatizer()
class LemmaTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmaTfidfVectorizer, self).build_analyzer()
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

vectorizer = LemmaTfidfVectorizer(ngram_range=(1, 2),
                             max_features=20000,
                             **tfidf_param)

In [ ]:
start_vect=time.time()
df_bag = vectorizer.fit_transform(df["News"])
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

In [ ]:
vocabulary = np.array(vectorizer.get_feature_names())
tfidf_sorted = np.argsort(df_bag.toarray()).flatten()[::-1]
print("Most Important Words in All Vectorization:\n",vocabulary[tfidf_sorted][:20])

## Save the output

In [ ]:
df.to_csv('ML_ready.csv')